# Let's import the necessary functions that we will use throughout the exercises

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML,display, clear_output
import ipywidgets as widgets
%matplotlib inline
%matplotlib notebook

# A new model: grain growth

The solidification of a material is a complex process, very difficult to model accurately. In the field of metallurgy it is important to understand how solidification happens, since this will impact the mechanical properties of the material. 

The following image shows 4, among the many, possible microscopic configurations of steel, after it has been cooled down in different conditions. Each one of them has different properties and is used for different purposes. Knowing exactly which configuration is growing when casting steel is important to obtain a material with specific properties.
<img src="images/ironPhases.png" width="1000" height="280" />

## Atoms and phases of matter

The smallest constitutive unit of matter that we consider for materials is the atom, which can, for simplicity, be approximated by a sphere. Atoms can arrange in various ways, and different phases can exist. Normally we talk about gas (atoms don't have any structure), liquid (atoms are disordered, but stay close to each other), and solid (ordered and packed).

<img src="images/statesofmatter.jpg" width="600" height="248" />

## Solid phase: crystals and grains

However, even when all of the atoms are in the same phase, they can still show some differences. Atoms in the solid phase can arrange themselves differently, as it can be seen in the following figure, where 3 common solid arrangements of atoms are shown:

<img src="images/scfccbcc.png" width="559" height="238" />

The same material, in different conditions, can show any of these (and many other!) configurations. Changes in chemical composition, pressure, temperature can drive the material from one structure to another. However, for the rest of the lecture, we will assume that all atoms have the same configuration (e.g. FCC) and not consider differences in this sense.

A group of atoms sharing the same arrangement and direction is called a <b>crystal</b>. Crystals are formed upon solidifications, when the atoms begin to re-arrange themselves in an ordered manner. However we talk about a "crystalline" system only when all of the atoms have the same orientation, which is hardly ever the case. It can happen that the atoms will have different orientations, different general directions, even if their internal arrangement is the same. In this case we talk about a polycristalline structure and we introduce the concept of grain.

<img src="images/crystal-poly.png" width="325" height="200" />

A <b>grain</b> is a single crystal surrounded by other crystals of the same type, but at different orientations. Multiple grains, each oriented in a different manner, form the solid as we know it. The orientation, form and dimension of the grains are controlled by many factors, such as the cooling rate (the velocity at which the temperature is lowered) and the amount of other elements in the material, and affects the mechanical properties of the solid.

In the following figure we can clearly see different grains, i.e. different groups of atoms that oriented in the same manner. The border between two crystals is called "grain boundary" and it is also important to know how it is shaped and the angle between different grains.
<img src="images/grains-HRTEM-2.png" width="454" height="287" />

Therefore it is necessary to study how different parameters can influence the microscopic picture, in order to drive the solidification towards the system with the most interesting mechanical properties.

## Solidification

Solidification is the process or re-arrangement of the atoms from a very chaotic state to a very compact and ordinated one.

At microscopic level it is very complex and till today remains one of the biggest challenges in computational materials science. There are two main concept to understand in order to make a reasonable model:

   - Solidification is an activated process
   - Solidification is driven by temperature
    

### Activated process

Solidification does not happen at the same time everywhere, as the atoms do not tend to spontaneously re-arrange in a solid form on their own. It is usually necessary to wait for an "event" which creates a <b>seed</b> for the solidification. What happens in experiments is that a very small part of the liquid becomes solid (either spontaneously or due to some type of interaction) and forms a (small) solid seed. After this, all the liquid attached to it will start to solidify as well.

<img src="images/GrainGrowth.gif" width="226" height="198" />

### Driven by temperature

It is normally said that a liquid, once it has reached the solidification temperature, will become solid. However, this is not completely true. What really happens is that the liquid can be cooled beyond the transition temperature, before starting to solidify.

The lower is the temperature of the liquid, the higher the probability that a seed will appear and will start to solidify together with its surrounding atoms. Therefore a seed that appears early will have more time to solidify the liquid, growing bigger than other grains.

## Modelling grain growth with a CA

Keeping in mind these two concepts, we can imagine what is necessary in order to provide a reasonable model of solidification, using a CA. We have to introduce a seed in the system, from which the solidification will have origin, at a temperature that is different for each seed. This will change the dimension and shape of the grain during its growth.

Grains differ one from the other in terms of relative orientation of the atoms and this will be highlighted by using different colors for each grain. In reality each one, with its own orientation, will grow only following a certain direction and shape, however in this simple model we are going to disregard that.

In this simple model we are going to assume that the temperature at which seeds appear at the border and in the center of the liquid can differ. This reflects the fact that the liquid can interact with external factors that will make the seed appear first in either region.

The grains will grow by "capturing" the neighboring cells. The velocity of growth depends on the local supercooling of the liquid, which keeps increasing over time (as the liquid gets colder and colder).

Differently from before, the CA does not have periodic boundary conditions and cells at the border will not interact with those at the other side.

# These functions are now given, all the rules have already been implemented.

In [ ]:
def makeMatrix(nx=100, ny=100, initialization="random", concentration=0.5):
    """This function is used to create a matrix with the desired configuration. The easiest example is to create 
    a matrix of single values through the numpy functions np.zeros or np.ones. Other initialization can be created
    at will, given a good implementation."""
    if initialization == "zeros":
        matrix = np.zeros((ny,nx))
    elif initialization == "random":
        matrix = np.random.rand(ny,nx)
        if 0 < concentration < 1:
            matrix = (matrix < concentration).astype(int)
        else:
            print("Concentration must be between 0 and 1")
    else:
        print("Inconnu")
        
    return matrix


def ruleIterategg(matrix, shadowMatrix, undercoolingMatrix, seedMatrix, timestep=0.001, ngrains=42): 
    newMatrix = matrix.copy()
    leny = matrix.shape[0]
    lenx = matrix.shape[1]
    
    neighborList = [[-1,0],[0,-1],[1,0],[0,1]]
    
    # Just consider the cells that are non zero, i.e. the solid ones
    grainy, grainx = matrix.nonzero()
    # Iterate over the non zero cells to propagate them
    for _idf in range(len(grainx)):
        idgrain = int(matrix[grainy[_idf],grainx[_idf]]-1)
        for delta in neighborList:
            [idx,idy] = (np.array([grainx[_idf],grainy[_idf]]) + np.array(delta))
            if (0 <= idx < lenx) and (0 <= idy < leny) and matrix[idy,idx] == 0:
                shadowMatrix[idy,idx,idgrain] += (undercoolingMatrix[grainy[_idf],grainx[_idf]]**3 \
                                                  + undercoolingMatrix[grainy[_idf],grainx[_idf]]**2)*timestep
                if shadowMatrix[idy,idx,idgrain] >=1:
                    newMatrix[idy,idx] = idgrain+1
                    seedMatrix[idy,idx] = 0
    
    # The non zero cells in the seed matrix
    seedy, seedx = seedMatrix.nonzero()
    # If now they are able to spawn, then make them spawn
    for _ids in range(len(seedx)):
        if undercoolingMatrix[seedy[_ids],seedx[_ids]] > seedMatrix[seedy[_ids],seedx[_ids]]:
            newMatrix[seedy[_ids],seedx[_ids]] = np.random.randint(1,ngrains+1)
            seedMatrix[seedy[_ids],seedx[_ids]] = 0

    return newMatrix, shadowMatrix

def RunAndShowgg(nx,ny,\
                 nGrainsSurface,deltaTSurfaceMean,\
                 nGrainsVolume,deltaTVolumeMean, \
                 undercoolingVelocity, niter=10):
    
    global shadowMatrix
    global undercoolingMatrix
    
    # We initialize the necessary matrixes that we are going to use in the program
    cellularAutomata = makeMatrix(nx,ny,initialization="zeros")
    seedMatrix = makeMatrix(nx,ny,initialization="zeros")
    undercoolingMatrix = makeMatrix(nx,ny,initialization="zeros")
    orientations = 42
    sigmaT = np.abs(undercoolingVelocity)**0.66
    
    # Here I am placing the seeds in random positions of the matrix
    # First we do on the surface
    for _idGrain in range(nGrainsSurface):
        if np.random.randint(2):
            idx = np.random.randint(0,nx-1)
            idy = np.random.choice([0,ny-1])
        else:
            idx = np.random.choice([0,nx-1])
            idy = np.random.randint(0,ny-1)
        if seedMatrix[idy,idx] > 0:
            seedMatrix[idy,idx] = np.min([seedMatrix[idy,idx],\
                                          np.random.normal(loc=deltaTSurfaceMean,scale=sigmaT)])
        else:            
            seedMatrix[idy,idx] = np.random.normal(loc=deltaTSurfaceMean,scale=sigmaT)
    
    # Then we do it in the bulk
    for _idGrain in range(nGrainsVolume):
        idx = np.random.randint(int(nx*.15),int(nx*.85))
        idy = np.random.randint(int(ny*.15),int(ny*.85))
        if seedMatrix[idy,idx] > 0:
            seedMatrix[idy,idx] = np.min([seedMatrix[idy,idx],\
                                         np.random.normal(loc=deltaTVolumeMean,scale=sigmaT)])
        else:            
            seedMatrix[idy,idx] = np.random.normal(loc=deltaTVolumeMean,scale=sigmaT)
        
    # Inizialization of the cellmatrix to guarantee that it spans all of the colourbar
    cellmatrix = makeMatrix(nx,ny,initialization="random")*orientations
    # Inizialization of the shadowMatrix that will contain the values of the 
    shadowMatrix = np.zeros((nx,ny,orientations+1))

    fig = plt.figure(figsize= (8,8))
    im = plt.imshow(cellmatrix,animated=True,cmap="Spectral")
    plt.title("Supercooling = {} K".format(np.min(undercoolingMatrix)))
    plt.colorbar()
    
    timestep = np.abs(1/(undercoolingVelocity*niter)**3)

    def runstep(i):
        global cellmatrix
        global shadowMatrix
        global undercoolingMatrix

        if i==0: # Starting frame
            cellmatrix = cellularAutomata.copy()
            
        else: # Everything else
            undercoolingMatrix -= undercoolingVelocity
            cellmatrix, shadowMatrix = ruleIterategg(cellmatrix,shadowMatrix,undercoolingMatrix,\
                                                     seedMatrix,timestep,ngrains=orientations)
        #im.title("Supercooling = {} K".format(np.min(undercoolingMatrix)))
        im.set_data(cellmatrix)
        return im


    ani = animation.FuncAnimation(fig, runstep, frames=niter, interval = 80, repeat=False)
    plt.close()
    return ani

## Now we can play around with the parameters that affect the simulation

A few examples of interesting configurations are here provided, in order to show some patterns that can be created using this model.

The parameters that can be freely modified are:

   - <b>Square side</b> --> the dimension of the grid. The bigger the grid, the longer the simulation will take to evaluate
   - <b>Number of steps</b> --> the number of iterations of the simulation. The longer it is, the more time it will take to evaluate
   - <b>Seeds on surface</b> --> the number of seeds that must be randomly placed on the border of the grid
   - <b>Seeds in bulk</b> --> the number of seeds that must be randomly placed in the center of the grid
   - <b>Seed Temp surface</b> --> how far can the temperature can go from the mean for the seeds on the border
   - <b>Seed Temp volume</b> --> how far can the temperature can go from the mean for the seeds in the center
   - <b>Undercooling velocity</b> --> the cooling ratio of the liquid
    
### As a starting point, you can run the simulation as it is given

In this case we set the parameters so that the seeds will appear first on the surface and only after in the center. This is a common scenario, because the seeds tend to form first where they are in contact with something solid (like the container where it is casted).

How do you think the grains will look like?

In [ ]:
side = widgets.IntSlider(
    min=40,
    max=200,
    step=1,
    description='Square side:',
    tooltip='Number of cells on the x axis',
    value=100,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

nGrainsSurface = widgets.IntSlider(
    min=40,
    max=200,
    step=1,
    description='Seeds on surface:',
    tooltip='Number of seeds on the border region',
    value=80,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

nGrainsVolume = widgets.IntSlider(
    min=40,
    max=200,
    step=1,
    description='Seeds in bulk:',
    tooltip='Number of seeds in the bulk region',
    value=60,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

deltaTSurfaceMean = widgets.FloatSlider(
    min=0.5,
    max=10,
    step=0.5,
    description='Seed Temp surface [K]:',
    tooltip='Undercooling temperature at which seeds will appear on the surface',
    value=0.5,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

deltaTVolumeMean = widgets.FloatSlider(
    min=0.5,
    max=10,
    step=0.5,
    description='Seed Temp volume [K]:',
    tooltip='Undercooling temperature at which seeds will appear in the volume',
    value=10,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

undercoolingVelocity = widgets.FloatSlider(
    min=-1,
    max=-0.1,
    step=0.1,
    description='Undercooling velocity [K/step]:',
    tooltip='Velocity at which temperature reduces over time',
    value=-0.1,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

maxiter = widgets.IntSlider(
    min=40,
    max=200,
    step=1,
    description='Number of steps:',
    tooltip='Number of steps that will be computed',
    value=120,
    style={'description_width': 'initial'}, layout=widgets.Layout(width='50%', min_width='350px'))

reset = widgets.ToggleButton(
    value=False,
    description='Reset',
    disabled=False,
    button_style='',
    tooltip='Description'
)

def reprint(args):
    #old_value = args
    #print args['name'] not in ['value', 'function_body']
    if type(args['new']) == bool:
        clear_output()
        print("It can take up to a minute to generate the simulation")
        ani = RunAndShowgg(side.value,side.value,nGrainsSurface.value,deltaTSurfaceMean.value,nGrainsVolume.value,
                   deltaTVolumeMean.value,undercoolingVelocity.value, niter=maxiter.value)
        clear_output()
        display(side,maxiter,nGrainsSurface, nGrainsVolume, deltaTSurfaceMean,deltaTVolumeMean,undercoolingVelocity,reset,HTML(ani.to_jshtml()))
        #HTML(ani.to_jshtml())
        #print(ani)
        #cellmatrix = makeMatrix(side.value,side.value)
        #plt.imshow(cellmatrix)
        #plt.show()

reset.observe(reprint)
ani = RunAndShowgg(side.value,side.value,nGrainsSurface.value,deltaTSurfaceMean.value,nGrainsVolume.value,
                   deltaTVolumeMean.value,undercoolingVelocity.value, niter=maxiter.value)

display(side,maxiter,nGrainsSurface, nGrainsVolume, deltaTSurfaceMean,deltaTVolumeMean,undercoolingVelocity,reset,HTML(ani.to_jshtml()))

What does the simulation look like? Can you compare it with the experimental images presented at the beginning?

### Now try to run different simulations:

Try moving the sliders, changing only the temperature at which seeds appear in the bulk and on the surface and compare them to the initial images.

- Set the sliders so that the seeds appear roughly at the same temperature
- Set the sliders so that the seeds appear first in the bulk

Then try different combinations of parameters and look what happens


#### Extra work:
If you have gotten here and want to do more, you could try to implement some basic post-processing

Go back to the notebook view and try to implement a function that counts how many squares have been occupied by grains with a particular orientation.